# Обучение модели devDV bot

Еще посмотреть:
 - https://github.com/Grossmend/DialoGPT , Статья на хабре https://habr.com/ru/company/icl_services/blog/548244/ , модель обученная на 40 млн.предложений пикабу
 - https://github.com/buvanenko/finetune_rugpt3 на школьных сочинениях

## Установка окружения

In [1]:
# Подключаем диск для сохранения моделей на нём. Если не выполнять, будет сохранять модели локально
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
G_DRIVE = '/content/gdrive/MyDrive/'

if not os.path.exists(G_DRIVE):
  DATA_PATH= 'devdv_gpt3_model'
else:
  DATA_PATH= os.path.join(G_DRIVE, 'devdv_gpt3_model')
  os.makedirs(DATA_PATH, exist_ok=True)

FileExistsError: ignored

In [3]:
%%writefile setup.sh
pip install transformers==4.4.2
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
cd apex
git reset --hard a651

pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [4]:
!sh setup.sh

     |████████████████████████████████| 2.0MB 4.0MB/s 
     |████████████████████████████████| 901kB 25.6MB/s 
     |████████████████████████████████| 3.3MB 35.8MB/s 
Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.11 MiB | 28.33 MiB/s, done.
Resolving deltas: 100% (5464/5464), done.
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-hcsiwt3a
Created temporary directory: /tmp/pip-req-tracker-btolq9e_
Created requirements tracker '/tmp/pip-req-tr

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
import random
import json

## Формируем датасет

In [8]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)

In [9]:
# Получаем данные экпорта сообщений в группах телеграм: `devDV.json` - группа https://t.me/devDV, `devdvAI.json` - группа https://t.me/devdvAI 
!wget https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/data/devDV.json https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/data/devdvAI.json

--2021-06-04 12:18:17--  https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/data/devDV.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15599165 (15M) [text/plain]
Saving to: ‘devDV.json.1’

devDV.json.1        100%[===================>]  14.88M  26.0MB/s    in 0.6s    

2021-06-04 12:18:18 (26.0 MB/s) - ‘devDV.json.1’ saved [15599165/15599165]

--2021-06-04 12:18:18--  https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/data/devdvAI.json
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 202884 (198K) [text/plain]
Saving to: ‘devdvAI.json.1’

devdvAI.json.1      100%[===================>] 198.13K  --.-KB/

In [10]:
# Загружаем данные экспортированные из телеграм
with open('devDV.json', "r") as f:
    devdv_data = json.load(f)
with open('devdvAI.json', "r") as read_file:
    devdv_ai_data = json.load(read_file)

In [11]:
# Склеиваем массивы текстов
def glue_and_clear(x):
  character_map = {ord(' '): ' ', ord('\n'): ' ', ord('\t'): ' ', ord('\r'): None}
  if type(x) == str:
    return x.translate(character_map)
  elif type(x) == list:
    if len(x) == 1 and x[0]['type'] in ['link', 'text_link']: # Фильтруем посты только с ссылкой
       return None
    res = ''
    for val in x:
      if type(val) == str:
        res += val.translate(character_map)
      elif val['type'] in ['mention', 'link', 'bold', 'hashtag', 'text_link', 'italic'] and val['text']:
        res += val['text'].translate(character_map)
    return res
  return None

messagesAI = list(filter(lambda s: s is not None, [glue_and_clear(x['text']) for x in devdv_ai_data['messages'] if x['type'] == 'message' and x['text']]))
messagesDV = list(filter(lambda s: s is not None, [glue_and_clear(x['text']) for x in devdv_data['messages'] if x['type'] == 'message' and x['text']]))
print('devDV', len(messagesDV),messagesDV[:10])
print('devdvAI', len(messagesAI),messagesAI[:10])

devDV 36263 ['"Ищем программиста на проект.  Знание React и/или Angular или большое желание его освоить. Но больше требования к чувству стиля и юзаби́лити!!!" возможно неполный день, от 20 000 до 50 000 руб. на руки обращаться к Бондарь Василию 😉 https://khabarovsk.hh.ru/vacancy/22241381', 'Доброго всем дня.  Ищу свободного верстальщика на короткий проект. Проект на bootstrap + PHP Если тут есть таковые - пишите в личку.', 'полное видео с hajs4 (почти 3 часа 😱), уже с таймлайнами! https://www.youtube.com/watch?v=X4YAbtBrPvM', 'это Алексей приступом мизантропии от меня заразился 😂', 'С чем связан такой серьезный приступ? )', 'сейчас как вывалю сюда кучу 💩', '😂😂😂 Вываливай ) Интересно )', 'Можно в приват )', 'кстати, я подготовил бумаги на регистрацию в минюст, надо собраться протокол подписать и устав обсудить, там есть несколько вопросов', 'Можно после мастеркласса']
devdvAI 345 ['Поздравляем с победой команду devdv на хактоне РЖД. Этой позитивной новостью и предлагаю начать группу.  Р

In [12]:
# Функция определения длины диалога
def get_length_param(text: str, tokenizer) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

# Формируем из пары сообщений диалога, в формат для обучения
def prepare_to_dialog_format(dialog: list, tokenizer) -> list:
    dataset = []
    for row in dialog:
        spk0_len = get_length_param(row[0], tokenizer)
        spk1_len = get_length_param(row[1], tokenizer)
        dialog_str = f'|0|{spk0_len}|{row[0]}|1|{spk1_len}|{row[1]}'
        dataset.append(dialog_str)
    return dataset

# Извлекаем цепочку сообщений, дерево диалогов не выстраиваем для простоты, 
# 
def get_dialog_chain(messages: list) -> list:
  dialog = []
  # Вариант 1. Пара последовательных сообщений perplexity = tensor(35.8258)
  # spk1 = [x for idx, x in enumerate(messages) if idx % 2 == 0]
  # spk2 = [x for idx, x in enumerate(messages) if idx % 2 != 0]
  # for post1, post2 in zip(spk1, spk2):
  #    dialog.append([post1, post2])

  # Вариант 2. делаем пары: миксуем первое и второе, второе и третье и так далее. С вероятностью 60-70% будет похоже на вопрос-ответ. - перплексия 17ть, качество плохое
  for idx, post in enumerate(messages):
    if idx + 1 >= len(messages):
      break
    dialog.append([post, messages[idx + 1]])
  return dialog

In [13]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
dataset = []
dv_dialog = get_dialog_chain(messagesDV)
dv_dataset = prepare_to_dialog_format(dv_dialog, tokenizer)
print(dv_dataset[:10])
dataset.extend(dv_dataset)
ai_dialog = get_dialog_chain(messagesAI)
ai_dataset = prepare_to_dialog_format(ai_dialog, tokenizer)
print(ai_dataset[:10])
dataset.extend(ai_dataset)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


['|0|3|"Ищем программиста на проект.  Знание React и/или Angular или большое желание его освоить. Но больше требования к чувству стиля и юзаби́лити!!!" возможно неполный день, от 20 000 до 50 000 руб. на руки обращаться к Бондарь Василию 😉 https://khabarovsk.hh.ru/vacancy/22241381|1|2|Доброго всем дня.  Ищу свободного верстальщика на короткий проект. Проект на bootstrap + PHP Если тут есть таковые - пишите в личку.', '|0|2|Доброго всем дня.  Ищу свободного верстальщика на короткий проект. Проект на bootstrap + PHP Если тут есть таковые - пишите в личку.|1|2|полное видео с hajs4 (почти 3 часа 😱), уже с таймлайнами! https://www.youtube.com/watch?v=X4YAbtBrPvM', '|0|2|полное видео с hajs4 (почти 3 часа 😱), уже с таймлайнами! https://www.youtube.com/watch?v=X4YAbtBrPvM|1|1|это Алексей приступом мизантропии от меня заразился 😂', '|0|1|это Алексей приступом мизантропии от меня заразился 😂|1|1|С чем связан такой серьезный приступ? )', '|0|1|С чем связан такой серьезный приступ? )|1|1|сейчас к

In [14]:
# Сохраняем датасет
TCV_FILE='dataset.tsv'
data = pd.DataFrame(dataset)
data.to_csv(TCV_FILE, header=False, index=False)
# data = pd.read_csv(TCV_FILE, sep=',',header=None)
print(data.shape)
data

(36606, 1)


,0
0,"|0|3|""Ищем программиста на проект. Знание Rea..."
1,|0|2|Доброго всем дня. Ищу свободного верстал...
2,"|0|2|полное видео с hajs4 (почти 3 часа 😱), уж..."
3,|0|1|это Алексей приступом мизантропии от меня...
4,|0|1|С чем связан такой серьезный приступ? )|1...
...,...
36601,|0|3|Попробовал еще раз пайчарм и очень разоча...
36602,|0|3|Потом решил глянуть прогресс vscode и вот...
36603,|0|2|Надо будет попробовать VS Code я не пробо...
36604,"|0|2|+1. Я тоже за VS Code. По крайней мере, е..."


In [15]:
# Формируем выборку обучающую и проверочную
train=data.sample(frac=0.8) #random state is a seed value
test=data.drop(train.index)
print('Длина обучающей', len(train), 'и проверочной выборки', len(test))
print('Пример обучающей выборки')
train.head()

Длина обучающей 29285 и проверочной выборки 7321
Пример обучающей выборки


,0
25996,|0|1|Ровно в 12.00|1|1|Мне нет 😂
33262,"|0|2|Там очень смешные фразочки типо ""российск..."
27420,"|0|2|Ну и да, mod_rewrite не к ночи будь помян..."
21793,"|0|2|Ну почему же. В ТОГУ есть ещё ""бизнес-инк..."
15706,|0|1|И я экономлю время на циклах прототипиров...


In [16]:
# Сохраняем данные для обучения
with open('train.txt', 'w') as f:
  for val in train.values:
        f.write(f'{val[0]}\n')
with open('valid.txt', 'w') as f:
  for val in test.values:
        f.write(f'{val[0]}\n')        

## Запускаем finetuning


In [17]:
# Скачиваем скрипт для обучения
# !wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py # В оригинальном ошибка свойства мак длины токенов.
!wget https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/pretrain_transformers.py

--2021-06-04 12:19:11--  https://raw.githubusercontent.com/akumidv/startup-khv-ai-study/main/06_Chatbots/4_ruGPT3small/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34383 (34K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.58K  --.-KB/s    in 0.006s  

2021-06-04 12:19:11 (5.85 MB/s) - ‘pretrain_transformers.py’ saved [34383/34383]



In [20]:
# Запускаем обучения, выходна модель сохраняется по адресу `comment_model`: храним только 2 чекпоинта, вычисялем точность послей каждой 1000 шагов, 15ть эпох вместо 5ти. Если нужно продолжить с чекпоинта - добавляем     --should_continue
!python pretrain_transformers.py \
    --output_dir=$DATA_PATH \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --save_steps 1000 \
    --save_total_limit 2 \
    --eval_all_checkpoints \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 15 \
    --block_size 2048 \
    --overwrite_output_dir

Streaming output truncated to the last 5000 lines.
Iteration:  11% 112/1012 [01:02<08:18,  1.81it/s]
Iteration:  11% 113/1012 [01:02<08:17,  1.81it/s]
Iteration:  11% 114/1012 [01:03<08:18,  1.80it/s]
Iteration:  11% 115/1012 [01:03<08:17,  1.80it/s]
Iteration:  11% 116/1012 [01:04<08:16,  1.80it/s]
Iteration:  12% 117/1012 [01:04<08:16,  1.80it/s]
Iteration:  12% 118/1012 [01:05<08:15,  1.80it/s]
Iteration:  12% 119/1012 [01:06<08:15,  1.80it/s]
Iteration:  12% 120/1012 [01:06<08:14,  1.80it/s]
Iteration:  12% 121/1012 [01:07<08:13,  1.80it/s]
Iteration:  12% 122/1012 [01:07<08:12,  1.81it/s]
Iteration:  12% 123/1012 [01:08<08:12,  1.81it/s]
Iteration:  12% 124/1012 [01:08<08:12,  1.80it/s]
Iteration:  12% 125/1012 [01:09<08:11,  1.81it/s]
Iteration:  12% 126/1012 [01:09<08:11,  1.80it/s]
Iteration:  13% 127/1012 [01:10<08:10,  1.81it/s]
Iteration:  13% 128/1012 [01:11<08:09,  1.81it/s]
Iteration:  13% 129/1012 [01:11<08:08,  1.81it/s]
Iteration:  13% 130/1012 [01:12<08:08,  1.80it/s]

In [29]:
SAVE_ARH_PATH=os.path.join(G_DRIVE, 'devdv_model.zip')
print('Сохраняем модель:', SAVE_ARH_PATH)
!zip $SAVE_ARH_PATH $DATA_PATH/*


devdv_gpt3_model
Сохраняем модель: /content/gdrive/MyDrive/devdv_model.zip
updating: content/gdrive/MyDrive/devdv_gpt3_model/added_tokens.json (stored 0%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/checkpoint-14000/ (stored 0%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/checkpoint-15000/ (stored 0%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/config.json (deflated 49%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/eval_results.txt (stored 0%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/merges.txt (deflated 76%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/pytorch_model.bin (deflated 16%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/special_tokens_map.json (deflated 72%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/tokenizer_config.json (deflated 66%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/training_args.bin (deflated 44%)
updating: content/gdrive/MyDrive/devdv_gpt3_model/vocab.json (deflated 74%)


## Проверка модели

In [30]:
# Если было обучение, то нужно пропустить. Если с нуля, можно скачать готовую модель
if not os.path.exists(DATA_PATH):
  #!gdown --id 1-2LSN65t8cOO4WW4lp1muYzzqqaaX2RQ # Модель на 5ти эпохах, пепрлексия 17ть
  #!unzip /content/devdv_model.zip
  !gdown --id 18rsFBPyEAhsRbjAKpFpO_NUPEhZQPUfh # Модель на 15ти эпохах, перплексия 9ть 
  !unzip /content/devdv_model15.zip
  DATA_PATH= 'devdv_gpt3_model'

In [47]:
# Загружаем модель
tokenizer = AutoTokenizer.from_pretrained(DATA_PATH)
model = AutoModelForCausalLM.from_pretrained(DATA_PATH)

In [48]:
#@title Параметры модели

# Поле 'Max length tokens generate' должно быть больше 8, но меньше 256
max_length = 256 #@param {type:"integer"}
# def 3 Поле 'No repeat ngram size' должно быть больше или равно 1, но меньше или равно 10.
no_repeat_ngram_size = 3 #@param {type:"integer"}
# def 100 Поле 'Top K' должно быть больше или равно 1, но меньше или равно 500. Текущее значение: 
top_k = 100 #@param {type:"integer"}
# def 0.9 Поле 'Top P' должно быть больше или равно 0.01, но меньше или равно 1.0.
top_p = 0.9 #@param {type:"number"}
# def 0.6 Поле 'Temperature' должно быть больше или равно 0.01, но меньше или равно 1.0. 
temperature = 0.6 #@param {type:"number"}
# def 5. Поле 'Num responses return' должно быть больше или равно 1, но меньше или равно 10. 
num_return = 5 #@param {type:"integer"}
# def True
do_sample = True  #@param {type:"boolean"}
# def True
is_always_use_length = True  #@param {type:"boolean"}
# Поле 'Length generate' должно принимать одно из следующих значений: [0, 1, 2, 3].
length_gen = 1  #@param {type:"integer"}
# def False. debug
log_debug = True  #@param {type:"boolean"}

params = { 
        "max_length": max_length,
        "no_repeat_ngram_size": no_repeat_ngram_size, 
        "top_k": top_k,                              
        "top_p": top_p,                              
        "temperature": temperature,                   
        "num_return_sequences": num_return,           
        "do_sample": do_sample,
        "device": 'cuda', #'cuda:0', # 'cpu',
        "is_always_use_length": is_always_use_length,
        "length_generate": length_gen, 
        "log_debug": log_debug
    }

In [49]:
# Генерация ответа
def generate(full_inputs, params): # inputs = [{'speaker': 0, 'text': 'Привет, как день прошел?'},{'speaker': 1, 'text': 'Хорошо, а у тебя как?'}]
  if len(full_inputs) > 5: # Ограничиваем кол-во истории в диалоге на 2 предыдущих + вопрос
    inputs = full_inputs[-5:]
  else:
    inputs = full_inputs
  inputs_text = ''
  for input_ in inputs:
    if params['is_always_use_length']:
        length_param = get_length_param(input_['text'], tokenizer)
    else:
        length_param = '-'
    inputs_text += f"|{input_['speaker']}|{length_param}|{input_['text']}"
  inputs_text += f"|1|{params['length_generate']}|"

  if params['log_debug']:
    print(f"        [debug] Затравка вопроса для модели: {inputs_text}")
  inputs_token_ids = tokenizer.encode(inputs_text, return_tensors='pt')

  try:
    # ToDo make this asynchronous
    outputs_token_ids = model.generate(
        inputs_token_ids,
        max_length=params['max_length'],
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=params['do_sample'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        temperature=params['temperature'],
        num_return_sequences=params['num_return_sequences'],
        device=params['device'],
        mask_token_id=tokenizer.mask_token_id,
        eos_token_id=tokenizer.eos_token_id,
        unk_token_id=tokenizer.unk_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )
  except Exception as e:
    print(f"===> Error generate: {str(e)}")
    return {'inputs': '', 'outputs': '', 'status': False, 'msg': f"{str(e)}"}

  outputs = [tokenizer.decode(x, skip_special_tokens=True) for x in outputs_token_ids]
  outputs = [x.split('|')[-1] for x in outputs]

  return {'inputs': inputs, 'outputs': outputs, 'status': True, 'msg': ''}

In [50]:
# Диалог с чатоботом
print('Введите quit для остановки')
if params['log_debug']:
  print(f"\n===> debug Params generate: {params}\n\n")
inputs = []
while True:
  user_input = input("USER:")
  if user_input == "quit":
    "stop talking!"
    break
  inputs.append({'speaker': 0, 'text': user_input})
  response_data = generate(inputs, params)
  bot_resposnse = response_data['outputs'][0]
  variants_responses = response_data['outputs']
  
  if not response_data['status']:
    print("BOT (ошибка):", response_data['msg'])
  else:
    bot_resposnse = response_data['outputs'][0]
    inputs.append({'speaker': 1, 'text': bot_resposnse})
    variants_responses = response_data['outputs']
    print("\n>>>BOT:", bot_resposnse)
    if params['log_debug']:
     print("        [debug] other variants ===> :", variants_responses[1:])
     

Введите quit для остановки

===> debug Params generate: {'max_length': 256, 'no_repeat_ngram_size': 3, 'top_k': 100, 'top_p': 0.9, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cuda', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


USER:Привет!


Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


        [debug] Затравка вопроса для модели: |0|1|Привет!|1|1|

>>>BOT: Ребзя, привет. Сегодня с утра иду знакомится с местными ИТ-компаниями.   Им нужны ИТ-шники на работу: разрабы, разовые проекты. Компания: Сбербанк. Нужно выполнить: разовые работы, нужно написать обоснование требований,  проектную документацию, нужно согласовать с руководством компании, нужны компетенции в части продвижения продукта.   Что нужно делать: написать письмо на @devdv_bot,
        [debug] other variants ===> : ['Всем привет!   Давно не виделись, но уже столько всего случилось!   Мы с @pewx2 решили немножко посидеть поболтать за жизнь, а заодно посмотреть видео доклад с DevFest Хабаровск.  В видео-интервью:  - как ребята и девченки делают свой DevFест в Хабаровске; - как и', '20 февраля в 13:00 в точке кипения «Грандиозное видео»  В видео-интервью с основателем и генеральным директором компании', 'Проект Start-UP Хабаровск  На площадке Региональной инновационной площадки ДВЖД по адресу ул. Фрунзе, д.42 пр

KeyboardInterrupt: ignored